# Scenario 1 - Kinetic Modeling
This Jupyter notebook will guide you through the process of visualizing and modeling your experimental data and finally uploading an EnzymeML document to a Database. The EnzymeML data format is based on the Systems Biology Markup Language (SBML) and helps with data consistency by storing all experimental data and the corresponding metadata in one place. <br>
A Juypter notebook contains text-cells, like this one, and code-cells, which you can run by pressing the small triangle on the top of the page or with the shortcut (CTRL+Return). Some code-cells will generate output directly under them. The current cell is highlighted by a blue line on the left. To choose a specific cell just click it. <br>
You can use this notebook as a step-by-step tutorial. For each step, a short text explains what will happen when you run the code-block and what you have to do. <br>
If not stated otherwise, please run each code-cell after you read the corresponding text.<br>
Before we start, we have to do some preparations in the next few cells. 

### Packages imports
Import packages used in this notebook

In [1]:
# API
import requests
import json
# Operating system for files
import os
# Jupyter widgets
import ipywidgets as widgets
from ipywidgets import Button, interact, interact_manual
# Tkinter for GUI
from tkinter import Tk, filedialog
# IPython display
from IPython.display import display, HTML
# Visualization
from string import Template

# loding apexcharts functionality
display(HTML('<script src="https://cdn.jsdelivr.net/npm/apexcharts"></script>'))

# global variables
workspace = ''
enzmldoc = ''

### Working directory
To have one place for all your data choose a directory. <br>
Run the next code cell and a button will appear. When you press the button a directory manager will open and you can navigate to a folder of your choice.

In [2]:
def setWorkspace(b):
    global workspace
    #button preparation
    out.clear_output()
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.path = filedialog.askdirectory()
    with out:
        if len(b.path) != 0:
            print('Workspace: ' + str(b.path))
            workspace = b.path
        else:
            workspace = None
            print('Workspace not set!')

buttonWorkspace = widgets.Button(
    description='Set Workspace',
    disabled=False,
    button_style='',
    tooltip='',
    icon=''
)
buttonWorkspace.on_click(setWorkspace)
out = widgets.Output()
display(buttonWorkspace,out)

Button(description='Set Workspace', style=ButtonStyle())

Output()

### Preparing the API
For convenient API-requests later on, some preparation has to be done.

In [3]:
api_url = "http://127.0.0.1:5000/"
# waiting for new API endpoint to convert excel to omex
endpoint_create = f"{api_url}/create"
endpoint_read = f"{api_url}/read"

## 1) EzymeML spreadsheet with experimental data
For your convenience, an EnzymeML spreadsheet template was designed, including tooltips for additional information. You can download a zip containing the empty Excel spreadsheet and an example of a filled spreadsheet [here](https://www.dropbox.com/s/ap26t13upye3cte/EnzymeML_Template_V1.zip?dl=1). <br> 
Please fill in the spreadsheet with your data and save it to the previously chosen directory. <br> 
If you have further questions or remarks, please contact us.

## 2) Convert the EnzymeML spreadsheet to an EnzymeML document
In this step the EnzymeML spreadsheet will be converted to an EnzymeML document. <br>
The EnzymeML document which is generated upon conversion is (1) a local .omex file or (2) a new entry in an EnzymeML dataverse <br>

2.1) Convert the EnzymeML spreadsheet to a local EnzymeML file in .omex format<br>
Run the next code cell to convert the spreadsheet to an EnzymeML document. An empty field and a button will appear. Enter the filename for the EnzymeML file in the empty field. Then press the button, from the window select the spreadsheet, and confirm. Now the spreadsheet will be automatically converted to an EnzymeML document via an API request. You can find the EnzymeML document with the file ending .omex in your working directory.<br>

In [4]:
# Some functions, may move those later on to the beginning or end of the notebook
def readBinary(inPath):
    # reads a binary file (like excel or .omex)
    if os.path.exists(inPath):
        return open(inPath, 'rb').read()

# MOCKUP - start
def readText(inPath):
    # reads a text or json file and teturns the content
    if os.path.exists(inPath):
        return open(inPath, 'r').read()
# MOCKUP - end

def writeBinary(inPath, inBinary):
    # writes a binary (like .omex)
    f = open(inPath, 'wb')
    f.write(inBinary)
    f.close()

def reqUnknown(excelBinary):
    # API request to create .omex from excel
    # add code as soon as it is implemented
    
    # MOCKUP - start
    # endpoint_create
    payload = readText('./datasets/API-Test-Fantasy.json')
    headers = {
      'Content-Type': 'text/plain'
    }

    response = requests.request("POST", endpoint_create, headers=headers, data=payload)
    return [response.status_code,response.content]
    # MOCKUP - ende

def reqRead(omexBinary):
    # API request to read EnzymeML returns EnzymeML document as json
    
    # endpoint_read
    # MOCKUP - start - omex is hardcoded
    payload={}
    files=[
        # ApPDC.omex
        ('omex',('ApPDC.omex',open('./datasets/Rother/ApPDC.omex','rb'),'application/octet-stream'))
        # Bommarius - Lagermann
        #('omex',('Lagermann.omex',open('./datasets/Bommarius/Lagermann/Lagermann_changed.omex','rb'),'application/octet-stream'))
        # 3IZNOK_TEST.omex
      #('omex',('3IZNOK_TEST.omex',open('./datasets/3IZNOK_TEST.omex','rb'),'application/octet-stream'))
    ]
    headers = {}
    # MOCKUP - end

    response = requests.request("GET", endpoint_read, headers=headers, data=payload, files=files)
    return [response.status_code,response.content]

def excel2omex(b):
    global enzmldoc
    # button preparation
    out.clear_output()   
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    excel = filedialog.askopenfilename(filetypes=[("Excel files", ".xls .xlsx .xlsm")],multiple=False)    # List of selected files will be set button's file attribute.
    
    # if user has not provided a name in the text field
    if len(textOmexName.value) == 0:
        with out:
            newName = os.path.basename(excel).split('.')
            newName.pop()
            textOmexName.value = '.'.join(newName)
    
    # add correct file ending
    if not textOmexName.value.lower().endswith('.omex'):
        textOmexName.value = textOmexName.value + '.omex'
        
    excelBinary = readBinary(excel)
    newOmex = reqUnknown(excelBinary)
    # check if API request worked as intended
    if newOmex[0] == 200:
        # write omex
        writeBinary(os.path.join(workspace, textOmexName.value), newOmex[1])
        with out:
            print('Omex created: '+ textOmexName.value)
            # directly read omex to get enzymeML document as json to work with
            reqJSON = reqRead(newOmex[1])
            if reqJSON[0] == 200:
                enzmldoc = json.loads(reqRead(newOmex[1])[1])
                print("EnzymeML document was loaded")
            else:
                with out:
                    print("data was not loaded correctly, Error: "+ str(reqJSON[0]))
    else:
        with out:
            print("Not Working")

In [5]:
# text field to enter EnzymeML name
textOmexName = widgets.Text(
    value='',
    placeholder='name.omex',
    description='OmexName:',
    disabled=False
)
# button to open directory, to select excel file
button = widgets.Button(
    description='Select Excel File',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon=''
)
out = widgets.Output()
if not workspace:
    with out:
        print('Workspace not set!')
button.on_click(excel2omex)
display(textOmexName,button,out)

Text(value='', description='OmexName:', placeholder='name.omex')

Button(description='Select Excel File', style=ButtonStyle(), tooltip='Click me')

Output()

2.2) Convert the EnzymeML spreadsheet to an EnzymeML dataverse <br>
Run the next code cell to convert the spreadsheet to an EnzymeML document. An empty field and a button will appear. Enter the address of the EnzymeML dataverse in the empty field. Then press the button, from the window select the spreadsheet, and confirm. Now the spreadsheet will be automatically converted to an EnzymeML document via an API request. You can find the EnzymeML document as new EnzymeML dataverse entry.<br>

In [ ]:
# later

## 3) Visualizing of time-course data
The next code cell will give you a basic overview of your reactions. Following this, you have the chance to dive deeper into your time-course data.

In [6]:
#Data-Preperation
def prepTable():
    reactions = enzmldoc['Reaction']
    proteins = enzmldoc['Protein']
    reactants = enzmldoc['Reactant']
    reaction_names=[]
    reaction_ids=[]
    reac_temp=[]
    reac_pH=[]
    educt_names={}
    prod_names={}
    mod_names={}
    prot_names={}
    for reac in reactions:
        reaction_names.append(reac['name'])
        reaction_ids.append(reac['id'])
        reac_temp.append(str(reac['temperature'])+' '+reac['tempunit'])
        reac_pH.append(7.5)
        educts=[]
        for educt in reac['educts']:
            for sub in reactants:
                if sub['id'] == educt['species']:
                    educts.append(sub['name'])
        educt_names[reac['id']]=educts
        products=[]
        for prod in reac['products']:
            for sub in reactants:
                if sub['id'] == prod['species']:
                    products.append(sub['name'])
        prod_names[reac['id']]=products
        modifiers=[]
        prots=[]
        for mod in reac['modifiers']:
            for sub in reactants:
                if sub['id'] == mod['species']:
                    modifiers.append(sub['name'])
            for p in proteins:
                if p['id'] == mod['species']:
                    prots.append(p['name'])
        mod_names[reac['id']]=modifiers
        prot_names[reac['id']]=prots
    row1=['Reactions']+reaction_names
    row2=['Temperature']+reac_temp
    row3=['pH']+reac_pH
    row4=['Educts']
    for reac in reaction_ids:
        educts = ''
        for educt in educt_names[reac]:
            if educts == '':
                educts = educts + educt
            else:
                educts = educts+ ', ' + educt
        row4.append(educts)
    row5=['Products']
    for reac in reaction_ids:
        products = ''
        for prod in prod_names[reac]:
            if products == '':
                products = products + prod
            else:
                products = products+ ', ' + prod
        row5.append(products)
    row6=['Proteins']
    for reac in reaction_ids:
        prots = ''
        for prot in prot_names[reac]:
            if prots == '':
                prots = prots + prot
            else:
                prots = prots+ ', ' + prot
        row6.append(prots)
    table = [row1,row2,row3,row4,row5,row6]
    return table

In [7]:
table = prepTable()
display(HTML('<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in table)
       )
))

Reactions,self-ligation of benzaldehyde_5_0
Temperature,576.3 K
pH,7.5
Educts,benzaldehyde
Products,benzoin
Proteins,pyruvate decarboxylase


Basic interactive Visualization

In [20]:
#Data-Preperation - static, has to change! 3IZNOK_TEST.omex
data = {}
data['series'] = []
data['xaxis'] = {}
for reaction in enzmldoc['Reaction']:
    for item in reaction['educts']:
        for replicate in item['replicates']:
            if len(data['xaxis']) == 0:
                data['xaxis']['categories'] = replicate['time']
            serie = {}
            serie['name'] = replicate['replica']
            serie['data'] = replicate['data']
            data['series'].append(serie)
print(data)

{'series': [], 'xaxis': {}}


In [11]:
#Data preparation - static, has to change! ApPDC.omex, Bommarius:Lagermann
data = {}
data['series'] = []
data['xaxis'] = {}
tmpYUnits=[]
tmpXUnits=[]
for reaction in enzmldoc['Reaction']:
    data['header'] = reaction['name']
    for item in reaction['products']:
        for replicate in item['replicates']:
            if len(data['xaxis']) == 0:
                data['xaxis']['categories'] = replicate['time']
            serie = {}
            serie['name'] = replicate['replica']
            serie['data'] = replicate['data']
            data['series'].append(serie)
            if not replicate['data_unit'] in tmpYUnits:
                tmpYUnits.append(replicate['data_unit'])
            if not replicate['time_unit'] in tmpXUnits:
                tmpXUnits.append(replicate['time_unit'])
if len(tmpYUnits) == 1:
    data['yAxisLabel'] = tmpYUnits[0]
else:
    data['yAxisLabel'] = 'Not the same concentration unit!'
if len(tmpXUnits) == 1:
    data['xAxisLabel'] = tmpXUnits[0]
else:
    data['xAxisLabel'] = 'Not the same time unit!'

In [12]:
#Insert in HTML Template
#HTML templet
html_template = Template('''
<div id="chart"></div>
<script> 
	var options = {
		series: $series,
		chart: {
			type: 'line',
			stacked: false,
			height: 350,
			zoom: {
				type: 'xy',
				enabled: true,
				autoScaleYaxis: true
			},
			toolbar: {
				autoSelected: 'zoom'
			}
		},
		dataLabels: {
			enabled: false
		},
		markers: {
			size: 0,
		},
		title: {
			text: '$header',
			align: 'center'
		},
		fill: {
			type: 'solid',
		},

		yaxis: {
			type: 'numeric',
			tickAmount: 7,
			labels: {
			formatter: function (val) {
				return (val).toFixed(4);
			},

			},
			title: {
				text: "$yAxisLabel",
			},
		},
		xaxis: {
			'categories': $xaxis
			,
			labels: {
				hideOverlappingLabels: true,
				rotate: 0,
			},
			title: {
				text: "$xAxisLabel",
			}
		},
		tooltip: {
			shared: false,
			x: {
			},
			y: {
				formatter: function (val) {
					return (val).toFixed(4)
				}
			}
		}
	};

	var chart = new ApexCharts(document.querySelector("#chart"), options);
	chart.render();

	function fToogleTooltip(){
		chart.updateOptions({
			tooltip: {
					shared: document.getElementById('toggleMergeTooltip').checked
			}
		})
	};
</script>
<div>
	<input type="checkbox" id="toggleMergeTooltip" onclick="fToogleTooltip()">
	<label for="toggleMergeTooltip">Multi Tooltip</label>
</div>
''')

The following graph is interactive, you can zoom or deselect time-series. With the Merge Tooltip checkbox you can toggle to show all series in the tooltip or only one. <br>
In the top-right corner you have the option to save the graph as png or svg.

In [13]:
display(HTML(html_template.substitute(series=data['series'], xaxis=data['xaxis']['categories'],header=data['header'], xAxisLabel=data['xAxisLabel'], yAxisLabel=data['yAxisLabel'])))

## 4) Kinetic modeling
Kinetic modeling consists of two steps: (1) Add a kinetic model to the EnzymeML document (2) Perform a parameter estimation using the selected kinetic model and the selected time course data in the EnzymeML document.<br>

4.1) Add a kinetic model to the EnzymeML document
Run the code cell with your selected model. You might add more than one model to the EnzymeML document. For consistency, we use [Systems Biology Ontology](http://www.ebi.ac.uk/sbo/main/) (SBO) Terms.

Irreversible Michaelis-Menten
[SBO:000029](http://www.ebi.ac.uk/sbo/main/SBO:0000029):

Michaelis-Menten with product inhibition
[SBO:0000387](http://www.ebi.ac.uk/sbo/main/SBO:0000387):

Reversible Michaelis-Menten
[SBO:0000438](http://www.ebi.ac.uk/sbo/main/SBO:0000438):

4.2) Perform a parameter estimation using the selected kinetic model and the selected time course data in the EnzymeML document <br>
Select the model and the time course data to be modeled, then use the COPASI API.

## 5) Visualizing the results
Visualize the kinetic model(s) from the previous step and compare the results.

## 6) Create a DataVerse entry
Run the next code cell to make a dataverse entry. A dropdown menu and a button will appear. Choose a dataverse. Then press the button, from the window select the EnzymeML document, and confirm. Now the EnzymeML document will be automatically transmitted via an API request. You can find the EnzymeML document as a new EnzymeML dataverse entry.<br>

## 7) Publish to SABIO-RK
Finally, publish your data to the SABIO-RK database. Press the button and select your EnzymeML document. After your confirmation, the EnzymeML document will automatically be validated and uploaded to the SABIO-RK database.